9. Write a program to predict the quality of water

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
upload=files.upload()

In [ ]:
data=pd.read_csv('water_dataX.csv',encoding="ISO-8859-1")
data.replace('NAN',0)


In [ ]:
data.dtypes

In [ ]:
data.columns=['station','location','state','temp','do','ph','con','bod','nn','fc','tc','year']
data

In [ ]:
#conversions
data['temp']=pd.to_numeric(data['temp'],errors='coerce')
data['do']=pd.to_numeric(data['do'],errors='coerce')
data['ph']=pd.to_numeric(data['ph'],errors='coerce')
data['bod']=pd.to_numeric(data['bod'],errors='coerce')
data['con']=pd.to_numeric(data['con'],errors='coerce')
data['nn']=pd.to_numeric(data['nn'],errors='coerce')
data['tc']=pd.to_numeric(data['tc'],errors='coerce')
data.dtypes

In [ ]:
data.shape

In [ ]:
data.columns=['station','location','state','temp','do','ph','con','bod','nn','fc','tc','year']
data

In [ ]:
data.tail()

In [ ]:
#calulation of Ph
data['npH']=data.ph.apply(lambda x: (100 if (8.5>=x>=7) else(80 if  (8.6>=x>=8.5) or (6.9>=x>=6.8) else(60 if (8.8>=x>=8.6) or (6.8>=x>=6.7) else(40 if (9>=x>=8.8) or (6.7>=x>=6.5) else 0)))))


In [ ]:
#calculation of dissolved oxygen
data['ndo']=data.do.apply(lambda x:(100 if (x>=6)
                                 else(80 if  (6>=x>=5.1)
                                      else(60 if (5>=x>=4.1)
                                          else(40 if (4>=x>=3)
                                              else 0)))))

In [ ]:
#calculation of total coliform
data['nco']=data.tc.apply(lambda x:(100 if (5>=x>=0)
                                 else(80 if  (50>=x>=5)
                                      else(60 if (500>=x>=50)
                                          else(40 if (10000>=x>=500)
                                              else 0)))))

In [ ]:
#calc of B.D.O
data['nbdo']=data.bod.apply(lambda x:(100 if (3>=x>=0)
                                 else(80 if  (6>=x>=3)
                                      else(60 if (80>=x>=6)
                                          else(40 if (125>=x>=80)
                                              else 0)))))

In [ ]:
#calculation of electrical conductivity
data['nec']=data.con.apply(lambda x:(100 if (75>=x>=0)
                                 else(80 if  (150>=x>=75)
                                      else(60 if (225>=x>=150)
                                          else(40 if (300>=x>=225)
                                              else 0)))))


In [ ]:
#Calulation of nitrate
data['nna']=data.nn.apply(lambda x:(100 if (20>=x>=0)
                                 else(80 if  (50>=x>=20)
                                      else(60 if (100>=x>=50)
                                          else(40 if (200>=x>=100)
                                              else 0)))))

data.head()
data.dtypes

In [ ]:

data['wph']=data.npH * 0.165
data['wdo']=data.ndo * 0.281
data['wbdo']=data.nbdo * 0.234
data['wec']=data.nec* 0.009
data['wna']=data.nna * 0.028
data['wco']=data.nco * 0.281
data['wqi']=data.wph+data.wdo+data.wbdo+data.wec+data.wna+data.wco
data

In [ ]:
#calculation overall wqi for each year
ag=data.groupby('year')['wqi'].mean()

In [ ]:
type(ag)
ag

In [ ]:
data=ag.reset_index(level=0)
type(data)

In [ ]:
data

In [ ]:
#using linear regression to predict
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
y = data['wqi']
x=data[['year']]

In [ ]:
reg=LinearRegression()
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=4)
print(x_train)
print(y_train)

In [ ]:
reg.fit(x_train,y_train)

In [ ]:
print(x_test)
a=reg.predict(x_test)
a

In [ ]:
y_test

In [ ]:
from sklearn.metrics import mean_squared_error
print('mse:%.2f'%mean_squared_error(y_test,a))

In [ ]:
#plotting the actual and predicted results
x_axis=data[['year']]
y_axis=data.wqi
y1_axis=reg.predict(x_axis)
plt.scatter(x_axis,y_axis)
plt.plot(x_axis,y1_axis,color='r')
plt.title("linear regression")
plt.show()
